In [1]:
## DATA
import torch

N = 100
T = 12

D1 = 5
D2 = 3
D3 = 8

V1 = torch.rand(N, T, D1) #view 1
V2 = torch.rand(N, T, D2) #view 2
V3 = torch.rand(N, D3) #view 3 with different shape

n_labels = 5
labels = torch.randint(n_labels, size=(N,1))

## Data Loader for training

In [2]:
class DataViews_torch(torch.utils.data.Dataset):
    def __init__(self, input_views: dict, target):
        super(DataViews_torch,self).__init__()
        self.target = target
        self.view_names = list(input_views.keys())
        self.views = [input_views[v_name] for v_name in self.view_names]
        
    def __len__(self):
        return len(self.views[0])
        
    def __getitem__(self, index):
        target = self.target[index]
        views = {view_n: view[index] for view, view_n in zip(self.views,self.view_names)}
        
        return {"index": index, "views": views, "target": target}

train_dataset = DataViews_torch(input_views={"view 1": V1, "view 2": V2, "view 3": V3}, target=labels)

train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=32,drop_last=False,shuffle=True)

train_dataset[2]

{'index': 2,
 'views': {'view 1': tensor([[0.6037, 0.9473, 0.6308, 0.3833, 0.7511],
          [0.5716, 0.5351, 0.4556, 0.5908, 0.2210],
          [0.7186, 0.5884, 0.0608, 0.8996, 0.9142],
          [0.3415, 0.4466, 0.3904, 0.7918, 0.4546],
          [0.4569, 0.2663, 0.4702, 0.4472, 0.3186],
          [0.0086, 0.8844, 0.8416, 0.5689, 0.2238],
          [0.3750, 0.0367, 0.6954, 0.1767, 0.8864],
          [0.8374, 0.7627, 0.4890, 0.6574, 0.5600],
          [0.9713, 0.1575, 0.9749, 0.4535, 0.3132],
          [0.2204, 0.8840, 0.4557, 0.9976, 0.6552],
          [0.7092, 0.4781, 0.3984, 0.4223, 0.6815],
          [0.5515, 0.0493, 0.9711, 0.7360, 0.5470]]),
  'view 2': tensor([[0.9491, 0.4606, 0.1015],
          [0.3673, 0.7667, 0.8475],
          [0.2531, 0.9373, 0.0419],
          [0.2623, 0.9134, 0.1321],
          [0.8031, 0.9152, 0.3755],
          [0.8543, 0.4214, 0.6850],
          [0.0835, 0.1835, 0.2483],
          [0.1269, 0.1443, 0.8095],
          [0.4972, 0.2590, 0.1855],
        

## Model encoder, prediction head and merge function

In [3]:
### MODEL
from mvlearning.single.models import create_model
from mvlearning.merge_module import MergeModule
from mvlearning.utils import get_dic_emb_dims
from mvlearning.fusion import FeatureFusion

DIM_EMBEDDING = 32

encoder_models = {}
for name, inp_dim in {"view 1": D1, "view 2":D2}.items():
    encoder_models[name] = create_model(inp_dim, DIM_EMBEDDING, model_type="lstm")
encoder_models["view 3"] = create_model(D3, DIM_EMBEDDING, model_type="mlp") #different model architecture

EMBEDDING_DIC= get_dic_emb_dims(encoder_models)
merge_function = MergeModule(EMBEDDING_DIC, mode="concat")

prediction_head = create_model(sum(EMBEDDING_DIC.values()), n_labels, model_type="mlp", encoder=False)

## Define MAUG (Sensor Dropout)

In [24]:
maug_arguments = {"maug": "sensd", "maug_args": {"drop_ratio":0.3}}

### Illustration 

In [37]:
from mvlearning.missing.simulate import augment_random_missing, augment_randomlist_missing

view_names = [ "S1", "S2","weather"]

print("Random augmented views from list = ",augment_randomlist_missing(view_names))
for _ in range (5):
    print(">> more random augmented views from list = ",augment_randomlist_missing(view_names))

print("Random augmented list with 10perc of dropout ratio = ",augment_random_missing(view_names, drop_ratio=0.10))
for _ in range(5):
    print(">> more random augmented list with 10perc of dropout ratio = ",augment_random_missing(view_names, drop_ratio=0.10))

print("Random augmented list with 10perc of dropout ratio = ",augment_random_missing(view_names, drop_ratio=0.90))
for _ in range(5):
    print(">> mode random augmented list with 10perc of dropout ratio = ",augment_random_missing(view_names, drop_ratio=0.90))

Random augmented views from list =  ['S1', 'S2']
>> more random augmented views from list =  ['weather']
>> more random augmented views from list =  ['S2']
>> more random augmented views from list =  ['S2', 'weather']
>> more random augmented views from list =  ['S2']
>> more random augmented views from list =  ['S1']
Random augmented list with 10perc of dropout ratio =  ['S1', 'S2', 'weather']
>> more random augmented list with 10perc of dropout ratio =  ['S1', 'S2', 'weather']
>> more random augmented list with 10perc of dropout ratio =  ['S2', 'weather']
>> more random augmented list with 10perc of dropout ratio =  ['S2', 'weather']
>> more random augmented list with 10perc of dropout ratio =  ['weather']
>> more random augmented list with 10perc of dropout ratio =  ['S2', 'weather']
Random augmented list with 10perc of dropout ratio =  ['S1']
>> mode random augmented list with 10perc of dropout ratio =  ['S2']
>> mode random augmented list with 10perc of dropout ratio =  ['weather'

# Training based on Pytorch

### Model Definition

In [25]:
## MODEL DEFINITION
mv_model = FeatureFusion(encoder_models, merge_function, prediction_head, **maug_arguments)

output_ = mv_model( {"view 1":V1, "view 2": V2, "view 3": V3})

print("Accuracy", torch.mean(((output_["prediction"] > 0.5) == labels).to(torch.float32)))

Accuracy tensor(0.1760)


In [26]:
## LOSS FUNCTION AND OPTIMIZER
optimizer = torch.optim.Adam(mv_model.parameters())
loss_function=torch.nn.CrossEntropyLoss()

loss_function(output_["prediction"], torch.squeeze(labels))

tensor(0.5194, grad_fn=<NllLossBackward0>)

### Train

In [27]:
EPOCHS = 50

for epoch in range(EPOCHS):
    
    mv_model.train(True)
    running_loss = 0.
    for i, data in enumerate(train_dataloader):
        optimizer.zero_grad()

        outputs = mv_model(data["views"])
        loss = loss_function(outputs["prediction"], torch.squeeze(data["target"]))
        loss.backward()

        optimizer.step()
        running_loss += loss
    print('EPOCH {}: with LOSS {}:'.format(epoch + 1, running_loss/len(train_dataloader)))

    ## VALIDATION/EARLY STOPPING IF REQUIRED

EPOCH 1: with LOSS 0.7880977392196655:
EPOCH 2: with LOSS 0.5458723306655884:
EPOCH 3: with LOSS 0.4586859345436096:
EPOCH 4: with LOSS 0.49492549896240234:
EPOCH 5: with LOSS 0.40600499510765076:
EPOCH 6: with LOSS 0.39157378673553467:
EPOCH 7: with LOSS 0.3639375567436218:
EPOCH 8: with LOSS 0.4806094765663147:
EPOCH 9: with LOSS 0.33303287625312805:
EPOCH 10: with LOSS 0.3622744083404541:
EPOCH 11: with LOSS 0.4265765845775604:
EPOCH 12: with LOSS 0.3206545114517212:
EPOCH 13: with LOSS 0.38373202085494995:
EPOCH 14: with LOSS 0.24350401759147644:
EPOCH 15: with LOSS 0.2805960774421692:
EPOCH 16: with LOSS 0.35794103145599365:
EPOCH 17: with LOSS 0.24888888001441956:
EPOCH 18: with LOSS 0.266275018453598:
EPOCH 19: with LOSS 0.20832347869873047:
EPOCH 20: with LOSS 0.22012090682983398:
EPOCH 21: with LOSS 0.3731854557991028:
EPOCH 22: with LOSS 0.19205619394779205:
EPOCH 23: with LOSS 0.18372194468975067:
EPOCH 24: with LOSS 0.17249898612499237:
EPOCH 25: with LOSS 0.141904279589653

In [28]:
output_ = mv_model( {"view 1":V1, "view 2": V2, "view 3": V3})

print("Accuracy", torch.mean(((output_["prediction"] > 0.5) == labels).to(torch.float32)))

Accuracy tensor(0.1760)


# Training based on Pytorch Lightning

### Model Definition

In [29]:
import pytorch_lightning as pl

## MODEL DEFINITION with LOSS FUNCTION and OPTIMIZER INSIDE
mv_model = FeatureFusion(encoder_models, merge_function, prediction_head, 
                         loss_function=torch.nn.CrossEntropyLoss(), optimizer="adam", **maug_arguments)

mv_model.configure_optimizers()

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [30]:
mv_model.loss_batch({
    "views": {"view 1":V1, "view 2": V2, "view 3": V3},
    "target": labels
})

{'objective': tensor(3.5678, grad_fn=<NllLossBackward0>)}

In [31]:
output_ = mv_model( {"view 1":V1, "view 2": V2, "view 3": V3})

print("Accuracy", torch.mean(((output_["prediction"] > 0.5) == labels).to(torch.float32)))

Accuracy tensor(0.1760)


### Train

In [32]:
trainer = pl.Trainer(max_epochs=50, accelerator="gpu", devices = 1)
trainer.fit(mv_model, train_dataloader, val_dataloaders=None)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params
-----------------------------------------------------
0 | views_encoder   | ModuleDict       | 431 K 
1 | merge_module    | MergeModule      | 0     
2 | prediction_head | Generic_Decoder  | 29.6 K
3 | loss_function   | CrossEntropyLoss | 0     
-----------------------------------------------------
461 K     Trainable params
0         Non-trainable params
461 K     Total params
1

Epoch 49: 100%|██████████| 4/4 [00:00<00:00, 114.53it/s, v_num=1, train_objective=0.0972] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 4/4 [00:00<00:00, 74.28it/s, v_num=1, train_objective=0.0972] 


In [33]:
output_ = mv_model( {"view 1":V1, "view 2": V2, "view 3": V3})

print("Accuracy", torch.mean(((output_["prediction"] > 0.5) == labels).to(torch.float32)))

Accuracy tensor(0.1780)
